# Energiedatenauswertung am Beispiel

Im folgenden Teil sollen Daten aus der Energiedatenerfassung ausgewertet werden.
Ein Beispieldatensatz wurde aus der Datenbank in die Datei "db_export_CMX600T_2022-12-01.csv" exportiert.

Lesen wir diese Datei zunächst ein. Anders als im letzten Beispiel sind die Einzelnen Spalten mit Semikolon getrennt.
Daher wird der sep-Parameter hier modifiziert:

In [ ]:
import pandas as pd
data = pd.read_csv("db_export_CMX600T_2022-12-01.csv", sep = ";")

In [ ]:
data.info()

Schauen wir uns die Daten noch einmal an, da hier kaum Zahlenwerte vorhanden sind:

In [ ]:
data

Auch die Zahlen sind hier in einem anderen Format, wir müssen die "," in "." wandeln.

In [ ]:
data = pd.read_csv("db_export_CMX600T_2022-12-01.csv", sep = ";", decimal=",")
data

Wir haben jetzt im Prinzip 2x einen Index. Der Originalindex ist in der Spalte ID eingetragen. Diesen können wir auch als Index für den DataFrame nutzen:

In [ ]:
data = data.set_index('ID')
data

###  <font color = darkblue> Aufgabe</font>
a) Erzeugen Sie einen neuen Dataframe, der nur den Zeitstempel_ms und die Wirkleistung_gesamt enthält!

In [ ]:
df = data[...]


b) Stellen Sie die Wirkleistung über der Zeit als Plot dar!

c) schauen Sie sich Bereiche an, z.B den ersten Peak.

### Einschub: Alternative Bibliothek für intreaktive Diagramme

Im Video aus dem Einstiegskurs wurde eine weitere Bibliothek vorgestellt, die eine interaktive Darstellung im Jupyter-Notebook ermöglicht. 
Es ist die altair-Bibliothek, auch diese Bibliothek ist im Python-Package-Index verfügbar:<br/>
https://pypi.org/project/altair/

Aber Achtung: die Bibliothek sorgt bei vielen Visualisierungen mit großen Datensätzen für sehr große Notebooks!

In [ ]:
pip install altair --user

In [ ]:
import altair as alt

df = data[['Zeitstempel_ms','Wirkleistung_gesamt']]

alt.data_transformers.disable_max_rows() # Eigentlich sind nur 5000 Zeilen erlaubt

alt.Chart(df).mark_line().encode(
    x='Zeitstempel_ms',
    y='Wirkleistung_gesamt'
).properties(
    width=800,
    height=300
).interactive()

### Ereignisse auswerten
Offensichtlich passiert in diesem Ausschnitt an einigen Stellen eine Änderung der Betriebszustände. 
Wir versuchen nur automatisiert auszuwerten, zu welchem Zeitpunkt die Ereignisse auftreten.<br/>


Erster Ansatz: Alle Punkte herausfiltern, die einen bestimmten Grenzwert überschreiten.

#### Werte über einem Grenzwert filtern

In [ ]:
df = data.loc[:,('Zeitstempel_ms','Wirkleistung_gesamt')]
df['Grenzwert'] = df[df.Wirkleistung_gesamt > 750].Wirkleistung_gesamt

number_of_samples = 6250
df = df[0:number_of_samples]

# Plot erzeugen
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df.Zeitstempel_ms, df.Wirkleistung_gesamt, linewidth=1.0, color='b')
ax.plot(df.Zeitstempel_ms, df.Grenzwert, linewidth=1.0, color='r')

# Plot anzeigen
plt.show()

###  <font color = darkblue> Aufgabe</font>

Filtern Sie alle Werte, die zwischen 600 und 750 liegen! Tipp: Zwei Bedingungen können mit & verknüpft werden!

In [ ]:
df = data.loc[:,('Zeitstempel_ms','Wirkleistung_gesamt')]


#### Gleitender Mittelwert als Filter

Oft ist es eine gute Idee, die Daten zunächst zu glätten. Das kann z.B. mit dem gleitenden Mittelwert passieren.

In [ ]:
df = data.loc[:,('Zeitstempel_ms','Wirkleistung_gesamt')]
df['Mittelwert'] = df['Wirkleistung_gesamt'].rolling(4).mean()

number_of_samples = 6250
df = df[0:number_of_samples]

# Plot erzeugen
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df.Zeitstempel_ms, df.Wirkleistung_gesamt, linewidth=1.0, color='b')
ax.plot(df.Zeitstempel_ms, df.Mittelwert, linewidth=1.0, color='r')

# Plot anzeigen
plt.show()

### Nutzung einfacher maschineller Lernalgorithmen zur Ausreißererkennung

#### Beispiel: Unüberwachte Ausreißererkennung unter Verwendung des lokalen Ausreißerfaktors (LOF).

Die Anomaliebewertung jeder Probe wird als lokaler Ausreißerfaktor bezeichnet. Er misst die lokale Abweichung der Dichte einer bestimmten Probe im Vergleich zu ihren Nachbarn. Er ist insofern lokal, als die Anomaliebewertung davon abhängt, wie isoliert das Objekt im Vergleich zu seiner Umgebung ist. Genauer gesagt ist die Lokalität durch die k-ächsten Nachbarn gegeben, deren Abstand zur Schätzung der lokalen Dichte verwendet wird. Durch den Vergleich der lokalen Dichte einer Probe mit den lokalen Dichten ihrer Nachbarn lassen sich Proben identifizieren, die eine wesentlich geringere Dichte als ihre Nachbarn aufweisen. Diese werden als Ausreißer bezeichnet.

Wir nutzen wieder eine Bibliothek. Diesmal scikit-learn eine Bibliothek für maschinelles Lernen:<br/>
https://pypi.org/project/scikit-learn/


In [ ]:
# Bibliothek importieren
from sklearn.neighbors import LocalOutlierFactor

# Wirkleistung umformatieren in einen Wertebereich zwischen -1 und 1
x = np.array(df['Wirkleistung_gesamt']).reshape(-1,1)

# Wir arbeiten mit den n-n*0.1 nächsten Nachbarn
model = LocalOutlierFactor(n_neighbors = int(number_of_samples - number_of_samples * 0.1))


label = model.fit_predict(x)
label

Alle Werte, die als Ausreißer identifiziert wurden sind mit -1 gelabelt. Plotten wir diese als Punkte:                                

In [ ]:
# Erzeuge 2 leere Listen
x = []
y = []

for i in range(len(label)):
    if label[i]!=1:
        x.append(df.iloc[i, 0])
        y.append(df.iloc[i, 1])

# Plot erzeugen
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df.Zeitstempel_ms, df.Wirkleistung_gesamt, linewidth = 1.0, color = 'b')
ax.scatter(x, y, color = 'r', marker = 'o')

# Plot anzeigen
plt.show()

###  <font color = darkblue> Aufgabe</font>
Ändern Sie die Außreißererkennung so ab, dass Sie immer in Abschnitten von 250 Samples nach Ausreißern suchen.
Es sollen also 25 Abschnitte untersucht werden.
Nutzen Sie das Code Template unten. Was beobachten Sie?

In [ ]:
df = data.loc[:,('Zeitstempel_ms','Wirkleistung_gesamt')]

number_of_samples = 250
# Das Modell müssen wir nur einmal erzeugen
model = LocalOutlierFactor(n_neighbors = int(number_of_samples - number_of_samples * 0.1))

for i in range(25):
    # Was macht diese Zeile?
    df1 = df[i * number_of_samples:(i * number_of_samples + number_of_samples)] 
    
    # Wirkleistung umformatieren in einen Wertebereich zwischen -1 und 1
    
    # Ihr Code...


###  <font color = darkblue> Aufgabe</font>

Vesuchen Sie nun mit Hilfe des Gelernten eine Auswertung zu schreiben, welche ermittelt, wenn die Tür zu lange geschlossen bleibt, ohne dass eine Bearbeitung gestartet wurde.